In [19]:
import pandas as pd
import numpy as np
import os
from os import walk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image

Use the 2018 task 3 validation data and validation ground truth for the following

Download Validation dataset: https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip

put the subset data in directory ./Data/ISIC2018_Task3_Validation_Input/

Downloade Saved Models from https://github.com/Avi-k-dua/Melanoma_Detection/tree/master/ISIC2018/Saved%20Models and put in directory ./Saved Models/

Download Validation Ground Truth: https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_GroundTruth.zip

In [21]:
# set the path to the 2018 validation images
test_path = os.path.normpath('./Data/ISIC2018_Task3_Validation_Input/')

# change to path for 'Saved Models'
ensemble_path = os.path.normpath('./Saved Models/')

In [4]:
# use the 2018 validation ground truth
df18_val = pd.read_csv('ISIC2018_Task3_Validation_GroundTruth.csv')

In [5]:
df18_val.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0034321,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0034322,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0034323,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,ISIC_0034324,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0034325,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df18_val_t = pd.melt(df18_val, id_vars=['image'],var_name='diagnosis', value_name='value')

df18_val_t = df18_val_t[df18_val_t['value'] == 1]
df18_val_t = df18_val_t.sort_values('image')
df18_val_t.diagnosis = df18_val_t.diagnosis.str.lower()
df18_val_t.reset_index(inplace = True, drop = True)

df18_val_t.rename(columns={'image': 'image_name', 'value': 'labels'}, inplace=True)
df18_val_t.labels = df18_val_t.diagnosis.map({
        'akiec': 0,
        'bcc': 1,
        'bkl': 2,
        'df': 3,
        'mel': 4,
        'nv': 5,
        'vasc': 6
    })

def append_ext(fn):
    return fn+".jpg"

df18_val_t.image_name = df18_val_t.image_name.apply(append_ext)

In [7]:
df18_val_t.head()

,image_name,diagnosis,labels
0,ISIC_0034321.jpg,nv,5
1,ISIC_0034322.jpg,nv,5
2,ISIC_0034323.jpg,bcc,1
3,ISIC_0034324.jpg,nv,5
4,ISIC_0034325.jpg,nv,5


In [8]:
df18_val_t.diagnosis.value_counts()

nv       123
bkl       22
mel       21
bcc       15
akiec      8
vasc       3
df         1
Name: diagnosis, dtype: int64

In [9]:
n = 10

reconstructed_ensemble1 = [0]*n
reconstructed_ensemble2 = [0]*n
reconstructed_ensemble3 = [0]*n

for k in range(n):
    reconstructed_ensemble1[k] = keras.models.load_model(os.path.join(ensemble_path , 'Ensemble1', f"Ensemble1_model{k}"))


for k in range(n):
    reconstructed_ensemble2[k] = keras.models.load_model(os.path.join(ensemble_path , 'Ensemble2', f"Ensemble2_model{k}"))


for k in range(n):
    reconstructed_ensemble3[k] = keras.models.load_model(os.path.join(ensemble_path , 'Ensemble3', f"Ensemble3_model{k}"))

Metal device set to: Apple M1


2022-03-24 16:32:24.102249: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-24 16:32:24.120951: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-24 16:32:24.129092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
batch_size = len(df18_val_t)
dataframe=df18_val_t

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=dataframe,
    directory=test_path,
    target_size=(128,128),
    x_col="image_name",
    y_col='diagnosis',
    batch_size=batch_size,
    shuffle=False,
    classes={
        'akiec': 0,
        'bcc': 1,
        'bkl': 2,
        'df': 3,
        'mel': 4,
        'nv': 5,
        'vasc': 6
        })

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Found 193 validated image filenames belonging to 7 classes.


## Predictions

Prediction of 2018 validation data set using 2018 ensembles

In [22]:
%%time
results1 = np.zeros( (test_generator.samples,7) ) 
results1
for j in range(n):
    test_generator.reset()
    results1 = results1 + reconstructed_ensemble1[j].predict_generator(test_generator, 
                                                           steps = STEP_SIZE_TEST)
# Performing Max Voting
results1 = np.argmax(results1,axis = 1)

print('ensemble 1')
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, results1))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(test_generator.classes, results1, target_names=target_names))

<timed exec>:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


ensemble 1
Confusion Matrix
[[  3   1   1   0   0   3   0]
 [  1  11   0   0   0   3   0]
 [  0   0   7   0   1  14   0]
 [  0   0   1   0   0   0   0]
 [  0   0   2   0   5  14   0]
 [  0   0   4   1   2 116   0]
 [  0   1   0   0   0   1   1]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.75      0.38      0.50         8
         bcc       0.85      0.73      0.79        15
         bkl       0.47      0.32      0.38        22
          df       0.00      0.00      0.00         1
         mel       0.62      0.24      0.34        21
          nv       0.77      0.94      0.85       123
        vasc       1.00      0.33      0.50         3

    accuracy                           0.74       193
   macro avg       0.64      0.42      0.48       193
weighted avg       0.72      0.74      0.71       193

CPU times: user 22.6 s, sys: 1.97 s, total: 24.5 s
Wall time: 26.6 s


In [24]:
%%time
results2 = np.zeros( (test_generator.samples,7) ) 
results2
for j in range(n):
    test_generator.reset()
    results2 = results2 + reconstructed_ensemble2[j].predict(test_generator, 
                                                           steps = STEP_SIZE_TEST)
# Performing Max Voting
results2 = np.argmax(results2,axis = 1)

print('ensemble 2')
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, results2))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(test_generator.classes, results2
                           , target_names=target_names
                           ))

ensemble 2
Confusion Matrix
[[  0   0   1   7   0   0   0]
 [  0   0   7   8   0   0   0]
 [  0   0   6  16   0   0   0]
 [  0   0   1   0   0   0   0]
 [  0   0   2  19   0   0   0]
 [  0   0  22 101   0   0   0]
 [  0   0   0   3   0   0   0]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.00      0.00      0.00         8
         bcc       0.00      0.00      0.00        15
         bkl       0.15      0.27      0.20        22
          df       0.00      0.00      0.00         1
         mel       0.00      0.00      0.00        21
          nv       0.00      0.00      0.00       123
        vasc       0.00      0.00      0.00         3

    accuracy                           0.03       193
   macro avg       0.02      0.04      0.03       193
weighted avg       0.02      0.03      0.02       193

CPU times: user 22.4 s, sys: 1.34 s, total: 23.8 s
Wall time: 24.4 s


/Users/leoliang/opt/anaconda3/envs/tf/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leoliang/opt/anaconda3/envs/tf/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leoliang/opt/anaconda3/envs/tf/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [25]:
%%time
results3 = np.zeros( (test_generator.samples,7) ) 
results3
for j in range(n):
    test_generator.reset()
    results3 = results3 + reconstructed_ensemble3[j].predict(test_generator, 
                                                           steps = STEP_SIZE_TEST)
# Performing Max Voting
results3 = np.argmax(results3,axis = 1)

print('ensemble 3')
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, results3))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(test_generator.classes, results3
                           , target_names=target_names
                           ))

ensemble 3
Confusion Matrix
[[  6   0   1   0   0   1   0]
 [  2  13   0   0   0   0   0]
 [  2   1  13   0   3   3   0]
 [  0   0   1   0   0   0   0]
 [  0   1   4   0  12   4   0]
 [  0   2  10   1   7 103   0]
 [  0   2   0   0   0   0   1]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.60      0.75      0.67         8
         bcc       0.68      0.87      0.76        15
         bkl       0.45      0.59      0.51        22
          df       0.00      0.00      0.00         1
         mel       0.55      0.57      0.56        21
          nv       0.93      0.84      0.88       123
        vasc       1.00      0.33      0.50         3

    accuracy                           0.77       193
   macro avg       0.60      0.56      0.55       193
weighted avg       0.80      0.77      0.77       193

CPU times: user 22.6 s, sys: 1.33 s, total: 23.9 s
Wall time: 24.5 s
